<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/ex12notebookB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVA2022 ex12notebookB

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/MVA/MVA-logo12.png"> https://www-tlab.math.ryukoku.ac.jp/wiki/?MVA/2022

----
## 演習問題 - 外食支出金額による県庁所在市のクラスタリング
----

主成分分析の講義と演習でも使った「SSDSE-家計消費」データでクラスタリングの実験をやってみましょう．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# SciPy の階層型クラスタリングパッケージ
import scipy.cluster.hierarchy as hierarchy

---
### データの準備

このセクションでは，この notebook で使うデータの準備をします．

独立行政法人統計センターが作成している 教育用標準データセット(SSDSE) の中の「SSDSE-家計消費」というデータを使います．

「SSDSE-家計消費」は，47都道府県の県庁所在市で調査された家計消費に関する226項目の数値から成るデータです．「二人以上の世帯の1世帯あたりの焼き肉（外食）への年間支出金額」みたいな数値が含まれています．

データの詳細はこちら: https://www.nstac.go.jp/sys/files/kaisetsu-C-2022.pdf

このセクションのコードの作成にあたっては，データの前処理の部分でこちらを参考にさせていただきました： https://studiolab.sagemaker.aws/import/github/icoxfog417/mlnote-note/blob/main/notebooks/chapter10_answer.ipynb

以下のコードを上から順に実行しましょう

In [ ]:
##### CSV ファイルを読み込む #####
URL = 'https://www.nstac.go.jp/sys/files/SSDSE-C-2022.csv'
df_raw = pd.read_csv(URL, encoding='cp932') # 文字コードが UTF8 ではなく ShiftJIS なので，encoding オプションを指定して変換
df_raw.head(10) # 最初の10行を表示

In [ ]:
#####  データの前処理 step1  #####

# df_raw の見出し行に記されたコードと，1行目に記されたその項目名との対応表を作り，項目名の行を削除
code = df_raw.iloc[0, :]
print('##### コードと項目名の対応')
print(code)
df1 = df_raw.drop(0, axis=0)

# 'SSDSE-C-2022'列の都市コードと'Prefecture'列の都道府県名，'City'列の県庁所在地名を整理
city = df1.iloc[:, :3]
print()
print('##### 都市コード，都道府県名，県庁所在地名')
print(city.head(5))
df1 = df1.drop(['Prefecture', 'City'], axis=1)
df1 = df1.set_index('SSDSE-C-2022')

df1 = df1.drop('R00000', axis=0) # 'R00000'列は「全国」のデータなのでこれを削除
df1 = df1.astype('float32') # 値を浮動小数点型に変換

print()
df1.head(5)

In [ ]:
#####  データの前処理 step2  #####

# 一人あたりの外食支出金額のデータを作成
df2 = pd.DataFrame()

# コードが 'LB12' で始まる列が，様々な品目の外食支出額．ただし，1世帯あたりの金額
columns = {}
for c in df1.columns:
    if c.startswith('LB12') and c != 'LB12':
        #print(c, code[c])
        columns[c] = code[c]
        df2[c] = df1[c]

# 列名をコードから項目名に変更
df2 = df2.rename(columns=columns)

# 都道府県ごとの平均世帯人数で割って一人あたりの金額にする
df2 = df2.divide(df1['LA03'], axis=0)

# 県庁所在地名の列を追加
df2 = pd.merge(city.loc[:, ['SSDSE-C-2022', 'City']], df2, left_on='SSDSE-C-2022', right_index=True)
df2 = df2.reset_index(drop=True)

df_feature = df2

# DataFrame 表示時の小数部の表示桁数
pd.options.display.precision = 1

# 最初の5件
#df_feature.head(5)

df_feature

In [ ]:
#####  データの前処理 step3  #####

# NumPy array にする
Xtmp = df_feature.drop(columns=['SSDSE-C-2022', 'City']).to_numpy()

# データを標準化
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
X = ss.fit_transform(Xtmp)
print(X.shape)
print(X[:5, :]) # 最初の5つのデータ

---
### 階層型クラスタリングしてみよう



上記の配列 `X` を学習データとして，階層型クラスタリングを実行します．
[scipy.cluster.hiearchy.linkage](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) でクラスタリングして，
[scipy.cluster.hierarchy.dendrogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.dendrogram.html) でデンドログラムを描きます．
標本間の距離はユークリッド距離，クラスタ間の距離はウォード法で測ります．

In [ ]:
link = hierarchy.linkage(X, method='ward', metric='euclidean')
fig, ax = plt.subplots(facecolor="white", figsize=(12, 6))
hierarchy.dendrogram(link, distance_sort='descending', show_leaf_counts=True, leaf_font_size=16, ax=ax)
plt.show()

以下のセルを実行すると，`threshold` で指定した値をクラスタ間距離のしきい値としてデータをクラスタに分けた結果を表示します． `label` がクラスタの番号に相当します（0ではなく1からはじまる番号）．

In [ ]:
threshold = 1.0 # クラスタ間距離のしきい値

label = hierarchy.fcluster(link, t=threshold, criterion='distance')
df_result = city.iloc[1:, :].copy()
df_result['label'] = label
df_result['idx'] = np.arange(47, dtype=int)
df_result = df_result.set_index('idx')
df_result = df_result.sort_values('label')
df_result[['label','Prefecture', 'City']]

### 問題1

次のことを考えて／調べてメモしておこう：

(1) この実験では何をやっている？どんな手法を使っている？

(2) 使っているデータのデータ数と次元数はいくつ？


### 問題2

次のことを考えて／調べてメモしておこう：

(1) 名古屋市とよく似てるのはどこだろう？ 2つ挙げよう．

(2) デンドログラムを見て，クラスタの数がそれぞれ 4, 6, 9 となるようなしきい値を選ぼう．それぞれのしきい値設定でクラスタリングした結果を観察しよう．

(3) (2) の結果から考えると，近畿5府県（滋賀，京都，大阪，兵庫，奈良）の県庁所在市の値は，互いに似てる？ 似てない？ それぞれのクラスタ数の条件で同じクラスタに入っているかどうか見てみよう．

(4) 47県庁所在市の中で，他の市との違いが大きくて最も孤立しているのはどこだろう？
孤立した点でなくなる（他のクラスタと合併される）ときのクラスタ間距離が最も大きい市を探そう．そこは何が特徴的なのだろう？
